In [12]:
from collections import defaultdict

import cv2
import pandas as pd
from sympy.physics.units import velocity
from ultralytics import YOLO
from pathlib import Path
import numpy as np

from src.models.metrics.metrics import calculate_metrics

In [32]:
current_file = Path().resolve()

src_dir = current_file
while src_dir.name != "src" and src_dir != src_dir.parent:
    src_dir = src_dir.parent
    
print(src_dir)

/Users/kobe/Documents/UCLL/School 2024-2025/AI Applications/ai-app-bot-brigade/src


In [33]:
yolo_pose_name = "yolo11s.pt"
yolo_pose_path = Path(src_dir / "data" / "weights" / "yolo_detection" / yolo_pose_name)

yolo_model = YOLO(yolo_pose_path)

In [34]:
def extract_features(track_data, bbox_w, bbox_h):
    area = bbox_w * bbox_h
    aspect_ratio = bbox_w / bbox_h
    
    area_change_avg = 0
    area_change_std = 0
    aspect_ratio_change_avg = 0
    aspect_ratio_change_std = 0
    aspect_ratio_avg = aspect_ratio
    aspect_ratio_std = 0
        
    P=min(10, len(track_data))
    
    if P>5:
        
        track_data_df = pd.DataFrame(track_data)
        
        track_data_last_p = track_data_df.iloc[-P:]
        
        col_area = 'area'
        col_ar = 'aspect_ratio'
        
        area_change_avg = track_data_last_p[col_area].dropna().diff().dropna().mean()
        area_change_std = track_data_last_p[col_area].dropna().diff().dropna().std()
        
        aspect_ratio_change_avg = track_data_last_p[col_ar].dropna().diff().dropna().mean()
        aspect_ratio_change_std = track_data_last_p[col_ar].dropna().diff().dropna().mean()
        
        aspect_ratio_avg = track_data_last_p[col_ar].mean()
        aspect_ratio_std = track_data_last_p[col_ar].std()
        
        

    return {'area': area, 
            'aspect_ratio': aspect_ratio, 
            'area_change_avg': area_change_avg, 
            'area_change_std': area_change_std, 
            'aspect_ratio_change_avg': aspect_ratio_change_avg, 
            'aspect_ratio_change_std': aspect_ratio_change_std,
            'aspect_ratio_avg': aspect_ratio_avg, 
            'aspect_ratio_std': aspect_ratio_std}

In [35]:
person_class_id = 0

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    track_history = defaultdict(list)
    collected_data = []

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (640, 360))

        results = yolo_model.track(resized_frame, persist=True)[0]
        
        xyxys = results.boxes.xyxy.int().tolist()           
                                                                        
        xywhs = results.boxes.xywh.int().tolist()         
                                                                  
        if results.boxes.id is not None:
            track_ids = results.boxes.id.int().tolist()   
        else:
            track_ids = []
        cls_ids = results.boxes.cls.int().tolist() 
        
        for xyxy, xywh, track_id, cls_id in zip(xyxys, xywhs, track_ids, cls_ids):
            if int(cls_id) == person_class_id:
                # xmin, ymin, xmax, ymax = xyxy
                x_av, y_av, bbox_w, bbox_h = xywh
                
                track_data = track_history[track_id]
                
                features = extract_features(track_data, bbox_w, bbox_h)
                
                track_data.append(features)
                
                collected_data.append({
                    "track_id": track_id,
                    "frame": cap.get(cv2.CAP_PROP_POS_FRAMES),
                    **features
                })
                
        frame_count += 1

    cap.release()
    return collected_data

In [47]:
video_name = 'simulation_chantier_2.mp4'
video_path = Path(src_dir / "local_movies" / "positive" / video_name)

In [48]:
data = process_video(video_path)


0: 384x640 1 car, 125.1ms
Speed: 1.8ms preprocess, 125.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 260.6ms
Speed: 0.8ms preprocess, 260.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 83.5ms
Speed: 1.1ms preprocess, 83.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 85.9ms
Speed: 1.4ms preprocess, 85.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 77.5ms
Speed: 1.4ms preprocess, 77.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 79.3ms
Speed: 0.8ms preprocess, 79.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 74.2ms
Speed: 1.3ms preprocess, 74.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 88.3ms
Speed: 1.9ms preprocess, 88.3ms 

In [49]:
df = pd.DataFrame(data)
df.to_csv("fall_detection_dataset.csv", index=False)